In [96]:
import numpy as np
import astropy.io.fits as pyfits
import pickle

In [97]:
data,header = pyfits.getdata('../spectrum_post_processing/galah_dr4_allspec_not_validated.fits', header=True)

In [98]:
description = dict()
# Major identifiers
description['sobject_id'] = 'GALAH identifier'
description['tmass_id'] = '2MASS identifier'
description['gaiadr3_source_id'] = 'Gaia DR3 source_id'
description['ra'] = 'propagated from Gaia DR3'
description['dec'] = 'propagated from Gaia DR3'

# Main spectroscopic results
description['flag_sp'] = 'Major spectroscopic fitting quality bitmask flag'
description['chi2_sp'] = 'Chi2 value of spectroscopic fitting'
description['model_name'] = 'Neural network model used for creating synthetic spectra'
description['teff'] = 'Spectroscopic effective temperature (used for fitting)'
description['e_teff'] = 'Uncertainty teff'
description['logg'] = 'Spectroscopic surface gravity (used for fitting)'
description['e_logg'] = 'Uncertainty logg'
description['fe_h'] = 'Abundance of Fe and all other elements not fitted in GALAH (Fe: 1D-NLTE)'
description['e_fe_h'] = 'Uncertainty fe_h'
description['flag_fe_h'] = 'Quality flag fe_h'
description['vmic'] = 'Microturbulence velocity (fitted)'
description['e_vmic'] = 'Uncertainty vmic'
description['vsini'] = 'Broadening velocity (fitted sme.vsini with sme.vmac=0)'
description['e_vsini'] = 'Uncertainty of vsini'

# Elemental abundances
for each in [
    'Li',
    'C',
    'N',
    'O',
    'Na',
    'Mg',
    'Al',
    'Si',
    'K',
    'Ca',
    'Sc',
    'Ti',
    'V',
    'Cr',
    'Mn',
    'Co',
    'Ni',
    'Cu',
    'Zn',
    'Rb',
    'Sr',
    'Y',
    'Zr',
    'Mo',
    'Ru',
    'Ba',
    'La',
    'Ce',
    'Nd',
    'Sm',
    'Eu',
]:
    description[each.lower()+'_fe'] = 'Elemental abundance for ['+each+'/Fe]'
    description['e_'+each.lower()+'_fe'] = 'Uncertainty '+each+'_fe'
    description['flag_'+each.lower()+'_fe'] = 'Quality bitmask flag of '+each+'_fe'

# Reduction information
description['v_bary_eff'] = 'Barycentric velocity applied to reduced spectra'
description['red_rv_ccd'] = 'Reduction pipeline best radial velocity for each CCD'
description['red_e_rv_ccd'] = 'Reduction pipeline uncertainty of red_rv_ccd'
description['red_rv_com'] = 'Reduction pipeline combined best radial velocity'
description['red_e_rv_com'] = 'Reduction pipeline uncertainty of red_rv_com'
description['red_teff'] = 'Reduction pipeline best teff'
description['red_logg'] = 'Reduction pipeline best logg'
description['red_fe_h'] = 'Reduction pipeline best fe_h'
description['red_alpha_fe'] = 'Reduction pipeline best alpha_fe'
description['red_vmic'] = 'Reduction pipeline best vmic'
description['red_vbroad'] = 'Reduction pipeline best vbroad'
description['red_flag'] = 'Reduction pipeline quality bitmask flag'

# Additional parameters of post-processing for binarity, emission, interstellar residuals etc.
description['sb2_rv_16'] = '16th perc. radial velocity of fit to syn-obs residuals'
description['sb2_rv_50'] = '50th perc. radial velocity of fit to syn-obs residuals'
description['sb2_rv_84'] = '84th perc. radial velocity of fit to syn-obs residuals'
description['ew_h_beta'] = 'Equivalent Width of fit for syn-obs residuals at Hbeta core'
description['ew_h_alpha'] = 'Equivalent Width of fit for syn-obs residuals at Halpha core'
description['ew_k_is'] = 'Equivalent Width of fit for K7699 Interstellar Line'
description['sigma_k_is'] = 'Sigma auf Gaussian fit for K7699 Interstellar Line'
description['rv_k_is'] = 'Radial velocity of fit to syn-obs residuals around K7699 line'
description['ew_dib5780'] = 'Equivalent Width of fit for 5780 Diffiuse Interstellar Band'
description['sigma_dib5780'] = 'Sigma auf Gaussian fit for 5780 DIB'
description['rv_dib5780'] = 'Radial velocity of fit to syn-obs residuals around 5780 DIB'
description['ew_dib5797'] = 'Equivalent Width of fit for 5797 Diffiuse Interstellar Band'
description['sigma_dib5797'] = 'Sigma auf Gaussian fit for 5797 DIB'
description['rv_dib5797'] = 'Radial velocity of fit to syn-obs residuals around 5797 DIB'
description['ew_dib6613'] = 'Equivalent Width of fit for 6613 Diffiuse Interstellar Band'
description['sigma_dib6613'] = 'Sigma auf Gaussian fit for 6613 DIB'
description['rv_dib6613'] = 'Radial velocity of fit to syn-obs residuals around 6613 DIB'

description['snr'] = 'Average signal-to-noise ratio (per pixel) of each CCD'

In [99]:
units = dict()
for each in description.keys():
    units[each] = ''
    if each in ['sobject_id','tmass_id','gaiadr3_source_id',
                'flag_sp','chi2_sp','model_name',
                'flag_fe_h',
                'red_flag',
                'snr'
               ]:
        pass
    elif each in ['teff','e_teff','red_teff']:
        units[each] = 'K'
    elif each in ['logg','e_logg','red_logg']:
        units[each] = 'log(cm.s**-2)'
    elif each in [
        'fe_h','e_fe_h','red_fe_h'
        ]:
        units[each] = 'dex'
    elif each in ['vmic','e_vmic','vsini','e_vsini',
                  'v_bary_eff',
                  'red_rv_ccd','red_e_rv_ccd','red_rv_com','red_e_rv_com',
                  'red_vmic','red_vbroad',
                  'sb2_rv_16','sb2_rv_50','sb2_rv_84','rv_k_is','rv_dib5780','rv_dib5797','rv_dib6613'
                 ]:
        units[each] = 'km s-1'
    elif each in ['ra','dec']:
        units[each] = 'deg'
    elif each[:6] == 'sigma_':
        units[each] = ''
    elif each[:3] == 'ew_':
        units[each] = 'Angstroem'
    elif each[-3:] == '_fe':
        units[each] = 'dex'
    else:
        print("'"+each+"',")

In [100]:
for each_index, each_key in enumerate(data.dtype.names):
    try:
        header['TTYPE'+str(each_index+1)] = each_key
        header['TUNIT'+str(each_index+1)] = units[each_key]
        header['TCOMM'+str(each_index+1)] = description[each_key]
    except:
        print(each_key)

tbhdu = pyfits.BinTableHDU.from_columns(columns=data,header=header)
tbhdu.writeto('galah_dr4_allspec_220713.fits')

In [101]:
schema = [
    ['*Column Name* | *Units* | *Description* | *Data Type* | '],
    [':--------------|:--------------|:------------|:----------|']
]
#star_id |  | 2MASS id   |char[16] | ]
for each_key in data.dtype.names:
    schema.append([each_key+' | '+str(units[each_key])+' | '+str(description[each_key])+' | '+str(data[each_key].dtype)+' | '])
np.savetxt('schema_galah_dr4_allspec',np.array(schema),fmt='%s')

In [102]:
a_file = open("../spectrum_post_processing/final_flag_sp_dictionary.pkl", "rb")
flag_sp_dictionary = pickle.load(a_file)
a_file.close()

schema = [
    ['*Bitmask* | *Description* | '],
    [':--------------|:--------------|']
]
schema.append(['0 | No flag rasied | '])
for key in flag_sp_dictionary.keys():
    schema.append([str(flag_sp_dictionary[key][0])+' | '+str(flag_sp_dictionary[key][1])+' | '])
np.savetxt('schema_galah_dr4_flag_sp',np.array(schema),fmt='%s')
schema

[['*Bitmask* | *Description* | '],
 [':--------------|:--------------|'],
 ['0 | No flag rasied | '],
 ['1 | Emission in Halpha/Hbeta detected | '],
 ['2 | Broadening (vsini) warning | '],
 ['4 | Microturbulence (vmic) warning | '],
 ['8 | chi square unusually low/high by 3 sigma | '],
 ['16 | Double line splitting detected (SB2) | '],
 ['32 | Not all 4 CCDs available | '],
 ['64 | Extrapolating spectrum model | '],
 ['128 | No spectroscopic analysis results available | ']]

In [103]:
schema = [
    ['*Bitmask* | *Description* | '],
    [':--------------|:--------------|']
]
schema.append(['0 | No flag rasied | '])
schema.append(['1 | Upper limit | '])
schema.append(['2 | No measurement available | '])
np.savetxt('schema_galah_dr4_flag_x_fe',np.array(schema),fmt='%s')
schema

[['*Bitmask* | *Description* | '],
 [':--------------|:--------------|'],
 ['0 | No flag rasied | '],
 ['1 | Upper limit | '],
 ['2 | No measurement available | ']]